In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
import glob
import cv2 as cv

This is the version of notebook which mount google drive on colab. So it is better to use the GPU of colab to train the CNN

In [ ]:
#Mounting google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#This function can be found in helpers.py
def png_to_mask(png):
    '''convert nd array with 0 and 1 values to png image, transparent for 0, color of choice for 1
    
    input
    -----
    png : 4 or 3 (without alpha) dimensionnal nd array of 0 and color values (or transparency) of uint8 (max 255)
    
    output
    ------
    nd numpy array with 0 for background and 1 for color pixels
    '''
    # NB opencv works with RGBA, so 4 dimensions, with A being alpha, the transparency (0=transparent)
    pngarray = np.array(png)
#     print(pngarray[:,:,0])
    mask = pngarray[:,:,0]
    mask[pngarray[:,:,0]!=0]=1
    mask[pngarray[:,:,1]!=0]=1
    mask[pngarray[:,:,2]!=0]=1
    if pngarray.shape[2]==4:
        mask[pngarray[:,:,3]!=0]=1
        
    return mask

In [ ]:
#Using glob to read the images
#On colab the names are non in alphabetic order so we sorted the names
imgs_names = glob.glob('/content/drive/MyDrive/th_analysedimages/*.tif')
imgs_names= sorted(imgs_names)
#imgs = [cv.imread(name, cv.IMREAD_UNCHANGED) for name in imgs_names[1]]
len(imgs_names)

In [ ]:
#Reading labels names
labels_names = glob.glob('/content/drive/MyDrive/labels/*.png')
labels_names= sorted(labels_names)
#labels = [png_to_mask(cv.imread(name, cv.IMREAD_UNCHANGED)) for name in labels_names]
len(labels_names)

In [ ]:
#Function to cut a image of 'cut1' and 'cut2' pixels 
#It shouldn't be necessary for our training on segmentation
def cut_y(img, cut1, cut2):
    new_img=img
    if cut1 != 0:
        start=int(np.trunc(cut1/2)) + (cut1 % 2)
        end= img.shape[0]-int(np.trunc(cut1/2))
        new_img=new_img[start:end,:]
    if cut2 != 0:
        start=int(np.trunc(cut2/2)) + (cut2 % 2)
        end= img.shape[1]-int(np.trunc(cut2/2))
        new_img=new_img[:,start:end]
    return new_img

In [ ]:
#Function for mirroring also this function is in helper.py
def extend_mirror(img, out_size):
    '''
    A method to extend an image to certain resolution by mirrorring the edges
    Input:
    :img: image as numpy array
    :out_size: a tuple of the desired output resolution
    Output:
    :out: the extended image
    '''
    # input error exceptions
    if np.any(img.shape>tuple(out_size)):
        raise Exception('Error: at least on of out_size axes is smaller than the image shape')
    if np.any(3*img.shape>tuple(out_size)):
        raise Exception('Error: at least on of out_size axes is at least 3 times larger than the image shape')
    # output parameters
    out = np.zeros(out_size)
    v_edge_u = (out_size[0]-img.shape[0]) // 2
    v_edge_d = -(out_size[0]-img.shape[0]-v_edge_u)
    h_edge_l = (out_size[1]-img.shape[1]) // 2
    h_edge_r = -(out_size[1]-img.shape[1]-h_edge_l)
    # output centre
    out[v_edge_u:v_edge_d,h_edge_l:h_edge_r] = img
    # output sides
    out[:v_edge_u,h_edge_l:h_edge_r] = np.flipud(img[:v_edge_u,:]) # top
    out[v_edge_d:,h_edge_l:h_edge_r] = np.flipud(img[v_edge_d:,:]) # bottom
    out[v_edge_u:v_edge_d,:h_edge_l] = np.fliplr(img[:,:h_edge_l]) # left
    out[v_edge_u:v_edge_d,h_edge_r:] = np.fliplr(img[:,h_edge_r:]) # right
    # output corners
    out[:v_edge_u,:h_edge_l] = np.fliplr(out[:v_edge_u,h_edge_l:h_edge_l*2]) # top-left
    out[:v_edge_u,h_edge_r:] = np.fliplr(out[:v_edge_u,2*h_edge_r:h_edge_r]) # top-right
    out[v_edge_d:,:h_edge_l] = np.fliplr(out[v_edge_d:,h_edge_l:h_edge_l*2]) # bottom-left
    out[v_edge_d:,h_edge_r:] = np.fliplr(out[v_edge_d:,2*h_edge_r:h_edge_r]) # bottom-right
    return out

Implementing U-net for our problem. Here all the 23 layers are used. Due to the fact that our problem consist in saying if a pixel is part or not of an air bubble the final output is given by tensor with two channels, each one corresponding to a matrix of dimensions equal to the original image. Using many convolutional layers the final output has a lower dimension than the input. In order to obtained the desired output the input image is expanded. The parts added are obtained mirroring parts of the real image as described in the paper. 
This tecnique is used in the training of NN.

In [ ]:
#Definining the architecture of U-net in pytorch
class U_net(nn.Module):
    def __init__(self, n_channels=64):
        'U-net from the paper "Olaf Ronneberger, Philipp Fischer, and Thomas Brox": https://arxiv.org/abs/1505.04597 '
        super(U_net, self).__init__()
        self.conv1 = nn.Conv2d(1,n_channels,3)
        self.conv2 = nn.Conv2d(n_channels,n_channels,3)
        self.pool = nn.MaxPool2d(2,stride=2)
        self.conv3 = nn.Conv2d(n_channels,2*n_channels,3)
        self.conv4 = nn.Conv2d(2*n_channels,2*n_channels,3)
        self.conv5 = nn.Conv2d(2*n_channels,4*n_channels,3)
        self.conv6 = nn.Conv2d(4*n_channels,4*n_channels,3)
        self.conv7 = nn.Conv2d(4*n_channels,8*n_channels,3)
        self.conv8 = nn.Conv2d(8*n_channels,8*n_channels,3)
        self.conv9 = nn.Conv2d(8*n_channels,16*n_channels,3)
        self.conv10 = nn.Conv2d(16*n_channels,16*n_channels,3)
        #self.upconv1= nn.ConvTranspose2d(16*n_channels,16*n_channels,2)
        self.upconv1 = nn.Upsample(scale_factor=2)
        self.conv11 = nn.Conv2d(16*n_channels,8*n_channels,3)
        self.conv12 = nn.Conv2d(8*n_channels,8*n_channels,3)
        #self.upconv2= nn.ConvTranspose2d(8*n_channels,8*n_channels,2)
        self.upconv2 = nn.Upsample(scale_factor=2)
        self.conv13 = nn.Conv2d(8*n_channels,4*n_channels,3)
        self.conv14 = nn.Conv2d(4*n_channels,4*n_channels,3)
        #self.upconv3= nn.ConvTranspose2d(4*n_channels,4*n_channels,2)
        self.upconv3 = nn.Upsample(scale_factor=2)
        self.conv15 = nn.Conv2d(4*n_channels,2*n_channels,3)
        self.conv16 = nn.Conv2d(2*n_channels,2*n_channels,3)
        #self.upconv4= nn.ConvTranspose2d(2*n_channels,2*n_channels,2)
        self.upconv4 = nn.Upsample(scale_factor=2)
        self.conv17 = nn.Conv2d(2*n_channels,n_channels,3)
        self.conv18 = nn.Conv2d(n_channels,n_channels,3)
        self.conv1_1= nn.Conv2d(n_channels,2,1)
        self.activation1=nn.ReLU()
    
    def forward(self, x):
        out = self.pool(self.activation1(self.conv2(self.activation1(self.conv1(x)))))
        out = self.pool(self.activation1(self.conv4(self.activation1(self.conv3(out)))))
        out = self.pool(self.activation1(self.conv6(self.activation1(self.conv5(out)))))
        out = self.pool(self.activation1(self.conv8(self.activation1(self.conv7(out)))))
        out = self.activation1(self.conv10(self.activation1(self.conv9(out))))
        out = self.upconv1(out)
        out = self.activation1(self.conv12(self.activation1(self.conv11(out))))
        out = self.upconv2(out)
        out = self.activation1(self.conv14(self.activation1(self.conv13(out))))
        out = self.upconv3(out)
        out = self.activation1(self.conv16(self.activation1(self.conv15(out))))
        out = self.upconv4(out)
        out = self.activation1(self.conv18(self.activation1(self.conv17(out))))
        out = self.conv1_1(out)
        return out


In [ ]:
#Accuracy
#The function select the 'correct' prediction (0 or 1) using argmax
def accuracy(pred, test_labels):
    '''
    pred: torch.tensor (result of U-net) of size [num_batches=1, 2, dim_image1, dim_image2]
    test_labels: torch.tensor (Real labels for the image) 
    '''
    '''
    Calculate the percentage of correct pixels labeled
    '''
    test_labels=test_labels.view(1,test_labels)
    label_pred=torch.argmax(pred,dim=1)
    acc= (torch.abs(label_pred-test_labels)).mean()
    return 1-acc 

In [ ]:
####Trying the training for one images
#This part is only to understand how GPU works on our U-net
#The traing is done on a single image
num_epochs=20
learning_rate=0.001
model=U_net()
optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate) #In the paper they use SGD
criterion=torch.nn.CrossEntropyLoss() 
for p in model.parameters():
    p.data = p.data.type(torch.cuda.DoubleTensor)
print("Starting training")
# If a GPU is available (should be on Colab, we will use it)
if not torch.cuda.is_available():
  raise Exception("Things will go much quicker if you enable a GPU in Colab under 'Runtime / Change Runtime Type'")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#Cycle for epochs
for epoch in range(num_epochs):
    # Train an epoch
    model.train()
    #Training using an image
    im= cv.imread(imgs_names[18], cv.IMREAD_UNCHANGED)
    
    print(imgs_names[18])
    y= png_to_mask(cv.imread(labels_names[18], cv.IMREAD_UNCHANGED))

    print(im.shape, y.shape)
    im=cut_y(im,33,99)
    y=cut_y(y,157,223)
    print(im.shape, y.shape)
    out_size=[572,572]
    #Mirroring like describe in the paper
    ext_x = extend_mirror(im, out_size)
    ext_x=torch.from_numpy(ext_x)
    ext_x=ext_x.view(1,1,572,572)
    ext_x=ext_x.to(device)
    y=torch.from_numpy(y) 
    y = y.type(torch.LongTensor)
    
    print(y.size())
    y=y.view(1,388,388)
    y=y.to(device)

    # Evaluate the network (forward pass)
    prediction = model(ext_x)
    loss = criterion(prediction,y)

    # Compute the gradient
    optimizer.zero_grad()
    loss.backward()

    # Update the parameters of the model with a gradient step
    optimizer.step()